## Усовершенствование ЭТАПА 1 (Разделение текста на чанки)

#### План

1. Обычный -  чанки по 1000 символов
2. С перекрытием - брать [1, 1000], [500, 1500]
3. Смысловой - обрезка текста только по предложениям и по смысловым блокам (в пределах заголовка, списков - в пользовательских соглашениях обычно так и пишут) 
4. Суммаризация каждого чанка для добавления в последующие 

![structure.png](../Schemes/Advanced_RAG.png)

## Библиотеки

In [1]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Splitting"

тестируем на 1 файле

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/Пользовательское соглашение Тинькофф/3_Пользовательское соглашение АО «Тинькофф Страхование».pdf")
docs = loader.load()

## Убрать лишние символы

In [3]:
# Убираем нумерацию страниц - replace("1 из 6", "")
for i in range(1, len(docs) + 1):
    text = docs[i-1].page_content.replace(str(i) + " из " + str(len(docs)), "")
    docs[i-1].page_content = text

## Мысль
записать все варианты чанков в файл и просто потом использовать готовую лучшую разбивку везде

## Вариант 1 
Берем чанки по 1800 символов

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1800)
splits = text_splitter.split_documents(docs)

In [6]:
splits[0]

Document(page_content='Пользовательское соглашение      АО «Тинькофф Страхование» (далее — Соглашение)     г. Москва      06 октября 2023 года     Настоящее Соглашение является публичной офертой и определяет условия использования материалов и сервисов, размещенных на официальном сайте АО «Тинькофф Страхование» (далее — Страховщик) в сети Интернет по адресу: tinkoﬃnsurance.ru, посетителями и пользователями данного интернет-сайта (далее — Сайт), а также условия и порядок дистанционного взаимодействия со Страховщиком с использованием Сайта и/или сайтов в сети Интернет страховых агентов и страховых брокеров Страховщика, а также аффилированных лиц Страховщика и/или их мобильных приложений.     1. Термины и определения     Договор страхования — договор, по которому одна сторона (Страховщик) обязуется за обусловленную договором плату (страховую премию) при наступлении предусмотренного в договоре события (страхового случая) осуществить другой стороне (страхователю) или иному лицу, в пользу кот

## Вариант 2
Берем чанки по 1800 символов с перекрытием 700

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1800, chunk_overlap=700)
splits = text_splitter.split_documents(docs)

print(len(splits), splits)

24 [Document(page_content='Пользовательское соглашение      АО «Тинькофф Страхование» (далее — Соглашение)     г. Москва      06 октября 2023 года     Настоящее Соглашение является публичной офертой и определяет условия использования материалов и сервисов, размещенных на официальном сайте АО «Тинькофф Страхование» (далее — Страховщик) в сети Интернет по адресу: tinkoﬃnsurance.ru, посетителями и пользователями данного интернет-сайта (далее — Сайт), а также условия и порядок дистанционного взаимодействия со Страховщиком с использованием Сайта и/или сайтов в сети Интернет страховых агентов и страховых брокеров Страховщика, а также аффилированных лиц Страховщика и/или их мобильных приложений.     1. Термины и определения     Договор страхования — договор, по которому одна сторона (Страховщик) обязуется за обусловленную договором плату (страховую премию) при наступлении предусмотренного в договоре события (страхового случая) осуществить другой стороне (страхователю) или иному лицу, в пользу

In [8]:
splits

[Document(page_content='Пользовательское соглашение      АО «Тинькофф Страхование» (далее — Соглашение)     г. Москва      06 октября 2023 года     Настоящее Соглашение является публичной офертой и определяет условия использования материалов и сервисов, размещенных на официальном сайте АО «Тинькофф Страхование» (далее — Страховщик) в сети Интернет по адресу: tinkoﬃnsurance.ru, посетителями и пользователями данного интернет-сайта (далее — Сайт), а также условия и порядок дистанционного взаимодействия со Страховщиком с использованием Сайта и/или сайтов в сети Интернет страховых агентов и страховых брокеров Страховщика, а также аффилированных лиц Страховщика и/или их мобильных приложений.     1. Термины и определения     Договор страхования — договор, по которому одна сторона (Страховщик) обязуется за обусловленную договором плату (страховую премию) при наступлении предусмотренного в договоре события (страхового случая) осуществить другой стороне (страхователю) или иному лицу, в пользу ко

## Вариант 3
Смысловая обрезка текста только по загловкам, если заголовок вместе с текстом больше 1000 чанков, то режется на несколько таких чанков по предложениям

In [2]:
import sys
sys.path.append('/Users/anastasia/Desktop/ML/Диплом')

import os
from my_library.format_docs import *
from my_library.summarization import *

[nltk_data] Downloading package punkt to /Users/anastasia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Путь к директории
directory = '../data/Пользовательское соглашение Тинькофф'

# Создаем пустой список для хранения всех путей файлов
file_paths = []

# Проходим по всем файлам и поддиректориям в директории
for root, directories, files in os.walk(directory):
    for filename in files:
        # Получаем полный путь к файлу
        filepath = os.path.join(root, filename)
        file_paths.append(filepath)

# получили пути
file_paths = file_paths[1:]

docs = []
for path in file_paths:

    # получаем разметку по словам
    info = extract_text_info(path)

    # собираем неполную разметку 
    formatted_text = process_text(info)
    # закрваем все теги 
    formatted_text = process_formatted_text(format_bold(formatted_text))
    
    docs.append(formatted_text)
    
    print(f"Загрузка файла... {os.path.basename(path)}:")

### Пример разметки html из PDF файла 
<h4>1 из 6</h4> <h1><b>Пользовательское соглашение АО «Тинькофф Страхование</b> (далее — Соглашение)</h1><h4>г. Москва 06 октября 2023 года Настоящее Соглашение является публичной офертой и определяет условия использования материалов и сервисов, размещенных на официальном сайте АО «Тинькофф Страхование» (далее — Страховщик) в сети Интернет по адресу: tinkoﬃnsurance.ru, посетителями и пользователями данного интернет-сайта (далее — Сайт), а также условия и порядок дистанционного взаимодействия со Страховщиком с использованием Сайта и/или сайтов в сети Интернет страховых агентов и страховых брокеров Страховщика, а также аффилированных лиц Страховщика и/или их мобильных приложений.</h4> <h2><b>1. Термины и определения</h2><h4>Договор страховани</b> — договор, по которому одна сторона (Страховщик) обязуется за обусловленную договором плату (страховую премию) при наступлении предусмотренного в договоре события (страхового случая) осуществить другой стороне (страхователю) или иному лицу, в пользу которого заключен договор (выгодоприобретателю), страховое возмещение в пределах определенной договором суммы (страховой суммы).

...

In [91]:
import json

# Путь к файлу, в который вы хотите сохранить массив строк
file_path = "docs_html.json"

# Записываем массив в файл в формате JSON
with open(file_path, "w") as file:
    json.dump(docs, file)

### Готовый вариант разбивания на чанки html документа
*<b>Плюсы</b>:* 
-  chunk_size
-  chunk_overlap
-  хранение в meta данных всех заголовков, которые имеют отношение к тексту

In [86]:
from langchain_text_splitters import HTMLHeaderTextSplitter

page_document = docs[0]
page_document = page_document.replace('<h4>', '<p>').replace('</h4>', '</p>')[15:]

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(page_document)


chunk_size = 1800
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(html_header_splits)
splits

[Document(page_content='Использование карты будет безопасным, если Вы постараетесь следовать простым правилам, приведенным в этом разделе. Пожалуйста, не забудьте также внимательно ознакомиться с разделом «Что такое ПИН-код и как обеспечить его конфиденциальность».• После получения карты поставьте свою подпись в прямоугольнике, расположенном на оборотной стороне карты. При отсутствии подписи карта может быть не принята к оплате в торгово-сервисном предприятии, а также повышается риск использования карты злоумышленниками в случае ее утраты/кражи.• Обращайтесь с картой также аккуратно и бережно, как и с наличными денежными средствами. Храните карту в безопасном месте, не оставляйте ее в автомобиле, в номере гостиницы, а также в местах, где посторонние имеют возможность взять карту, скопировать информацию, указанную на карте.• Никогда не принимайте рекомендации, советы, помощь от третьих лиц при пользовании картой, особенно при снятии наличных денежных средств в банкоматах. В случае возни

### Собственный вариант разбивания на чанки html документа
*<b>Плюсы</b>:* 
-  смысловая разбивка по предложениям
-  Фича с суммаризациия заголовков для передачи в следующий чанк

### Получаем чанки по n токенов

In [58]:
all_chunks = []
n = 200 

for doc in docs:
    current = split_into_chunks(doc)
    all_chunks += merge_chunks(current, n)

In [60]:
all_chunks

['Меры безопасности при использовании карты.  Использование карты будет безопасным, если Вы постараетесь следовать простым правилам, приведенным в этом разделе Пожалуйста, не забудьте также внимательно ознакомиться с разделом «Что такое ПИН-код и как обеспечить его конфиденциальность» • После получения карты поставьте свою подпись в прямоугольнике, расположенном на оборотной стороне карты При отсутствии подписи карта может быть не принята к оплате в торгово-сервисном предприятии, а также повышается риск использования карты злоумышленниками в случае ее утраты/кражи • Обращайтесь с картой также аккуратно и бережно, как и с наличными денежными средствами Храните карту в безопасном месте, не оставляйте ее в автомобиле, в номере гостиницы, а также в местах, где посторонние имеют возможность взять карту, скопировать информацию, указанную на карте • Никогда не принимайте рекомендации, советы, помощь от третьих лиц при пользовании картой, особенно при снятии наличных денежных средств в банкома

In [ ]:
import json

# Путь к файлу, в который вы хотите сохранить массив строк
file_path = "smart_chunks_50.json"

# Записываем массив в файл в формате JSON
with open(file_path, "w") as file:
    json.dump(all_chunks, file)

Сохраняем из-за внутренних конфликтов библиотек

In [ ]:
# Загружаем массив из файла
import json

file_path = "output_50.json"
with open(file_path, "r") as file:
    all_chunks = json.load(file)

# Выводим загруженный массив строк
print(len(all_chunks))

In [ ]:
for i, elem in enumerate(all_chunks):        
    print(i, elem)

### Пример смысла суммаризации 
Взятие часто встречающихся слов - дало краткое описание длиного заголовка и сэкономило пару слов, при этом не потеряв их смысл

In [ ]:
print(all_chunks[40])
print(all_chunks[41])

## Вариант 4
Процесс сумаризации текстов для добавления в чанки, для обработки запросов. Возможно также сильно сжимать текст -  для того, чтобы обозначить о чем вообще говориться в документах

варианты суммаризации - использовать llm или просто часто встречаемые слова во фразу переделывать

Буду использовать - библилотеку nltk для суммаризации - смысл взятие часто встречаемых слов

In [ ]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
import ssl
import nltk

# Деактивируйте проверку сертификатов в NLTK 
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download("punkt")  # Добавлено для загрузки ресурса "punkt"

# Загрузка списка стоп-слов для русского языка
stop_words = set(stopwords.words("russian"))

# Текст для суммаризации
text = """
Перечень документов и сведений, предоставляемых Клиентом (представителем Клиента) для присоединения к условиям Универсального договора, а также в целях обслуживания в рамках Универсального договора (далее — Перечень)
"""

# Регулярное выражение для токенизации предложений и фильтрации слов
sentence_splitter = re.compile(r'(?<!\d)\.(?! \d)|\.(?![a-z])')
word_filter = re.compile(r'[^\W\d]+')

# Токенизация текста на предложения
sentences = sentence_splitter.split(text)

# Токенизация предложений на слова и удаление стоп-слов
words = [word for sentence in sentences for word in word_tokenize(sentence) if word_filter.match(word) and word.lower() not in stop_words]

# Подсчет частоты слов
word_freq = Counter(words)

# Вывод самых часто встречающихся слов и их частоты
print(word_freq.most_common(5))

top_words = word_freq.most_common(3)

# Формирование краткой суммаризации на основе этих слов
summary = ' '.join([word[0] for word in top_words])

print(summary)